In [7]:
import pickle
import pandas

# read python dict back from the file
pkl_file = open('etf_df.pkl', 'rb')
dataframe2 = pickle.load(pkl_file)
pkl_file.close()

etf_

            iShares MSCI Finland Capped  iShares MSCI Denmark Capped  \
date                                                                   
2015-01-01                        32.81                        47.32   
2015-02-01                        34.50                        51.03   
2015-03-01                        33.75                        52.50   
2015-04-01                        33.43                        55.15   
2015-05-01                        33.78                        55.57   
2015-06-01                        32.50                        53.80   
2015-07-01                        33.61                        56.43   
2015-08-01                        31.49                        53.98   
2015-09-01                        30.36                        53.04   
2015-10-01                        32.81                        52.98   
2015-11-01                        33.03                        55.13   
2015-12-01                        32.61                        5